In [ ]:
#Updated on 6/28
#All monitoring decisions are included in the MP
#The coefficients of each arc in the MP is exact
#The path with the least detection probability = most probability of being undetected is approximate
using JuMP
using Gurobi
using LightGraphs
const gurobi_env = Gurobi.Env()
function solveMP()

    println("Done loading packages")
    # const MOI = Gurobi.MOI
    
    # setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0) #,LazyConstraints=1)
    println("Done setting param")
    # outfile = "C:/Users/din/Documents/GitHub/UncertainTarget/Log_Test_Output.csv"
    #     io = open(outfile, "w")
    # println(io,"Instance \tPath \tExact \tApprox")
    # global p_exact = []
    # global p_approx = []


    ins = 0 #:10

    include("./Ins"*string(ins)*".jl")
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    numArcs = length(arcs[:,1])
    X_feas = EnumX(arcs, b_x, numArcs, d_x)
    EnumY = EnumX(arcs, b_y, numArcs, d_y)
    println("EnumY = ", EnumY)
    M_set = [EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    numY = 1 
    R = 10^6
    P_set = [P[1]]

    # global x_now, current_Obj, y_now
    println("X_feas = ", X_feas)
    println("Starting M_set ", M_set, ", P_set = ", P_set)
    Y_sol = []

    numPaths = 1
    newP = true
    newM = true
    iter = 0
    tol = 10^(-6)
    theta_now = -1e12
    inner_Obj = 1-1e12 #set this to the smallest possible value of the obj val
    MP = Model(() -> Gurobi.Optimizer(gurobi_env))
    
    #Heuristics=0.0, Cuts = 0, OutputFlag = 0

    #MASTER PROBLEM
    # MP = direct_model(Gurobi.Optimizer(gurobi_env))
    @variable(MP, x[1:numArcs], Bin)
    @variable(MP, theta >= -1e12)
    @objective(MP, Min, theta)
    @constraint(MP, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)

    println(MP)
    # while outer_Obj < inner_Obj - tol #newP == true || newM == true
    #     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
    #     global newP, newM, P_set, M_set, numPaths,numY, iter, outer_Obj, inner_Obj
    println(iter)
    #     cb_calls = Int32[]
    #     cb_calls = Cint[]
    function my_callback_function(cb_data) #, cb_where::Cint)
    #     function my_callback_function(cb_data::Gurobi.CallbackData, cb_where::Int32)
#         global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#         global P_set, M_set, numPaths,numY, iter #, theta_now, inner_Obj
    #         global iter, P_set, M_set#, lambda, pi_, inner_Obj
    #         global iter_num

#         global iter
        iter += 1
        println("\nIteration number = ", iter)
        println(MP)
        x_now = callback_value.(Ref(cb_data), x)
        theta_now = callback_value.(Ref(cb_data), theta)

        println("\tx_now = ", x_now)
        println("\ttheta_now = ", theta_now)
        newP = false
        numPaths = length(P_set)
        numY = length(M_set)

        println("\tSolving SubProblem")
    #     println("\tPre-solve:P_set = ", P_set, ", M_set = ", M_set)
        P_set, M_set, lambda, pi_, inner_Obj = solveLP_findM(x_now)
        println("\tPost-solve:P_set = ", P_set, ", M_set = ", M_set)
        println("\tlambda = ", lambda, "; pi_ = ", pi_)

        numPaths = length(P_set)
        numY = length(M_set)
    #         dual_Obj = sum(lambda[i]*(-R)*sum(x_now[a] for a in P_set[i]) for i=1:numPaths)+pi_
    #         println("dual ", dual_Obj)
        TOL = 1e-6
        println("theta_now = ", theta_now, " vs subProb = ", inner_Obj)
        if theta_now + TOL >= inner_Obj  #fs_x_current ≈ fm_current # we are done
            @info("No additional constraint from the subproblem")
        end

        if theta_now + TOL < inner_Obj 
            println("Adding constraint")
            con = @build_constraint(theta - sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) >= pi_)
            println(" ", con)
    #         display(con)
            MOI.submit(MP, MOI.LazyConstraint(cb_data),con)
        end
        if iter > 3
            # You can terminate the callback as follows:
            GRBterminate(backend(MP))
        end
    end

    MOI.set(MP, MOI.RawParameter("LazyConstraints"), 1)
#     MOI.set(MP, Gurobi.CallbackFunction(), my_callback_function)
    MOI.set(
        MP,
        MOI.LazyConstraintCallback(),
        my_callback_function,
    )

    optimize!(MP)
end

solveMP()


In [ ]:
using JuMP, Gurobi, Test, Combinatorics, LightGraphs

# function solveMP()
    ins = 1 #:10
    include("./Ins"*string(ins)*".jl")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY
    
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    
    global numArcs = length(arcs[:,1])
    global X_feas = EnumX(arcs, b_x, numArcs, d_x)
    global EnumY = EnumX(arcs, b_y, numArcs, d_y)
    global M_set = [EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    global numY = 1 
    global R = 10^6#0^6
    global P_set = [P[1]]

#     global x_now, current_Obj, y_now
    println("X_feas = ", X_feas)
    println("Starting M_set ", M_set, ", P_set = ", P_set)
    global Y_sol = []

    global numPaths = 1
    global newP = true
    global newM = true
    global iter = 0
    global tol = 10^(-6)
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
    
    model = direct_model(Gurobi.Optimizer())
    set_optimizer_attribute(model, "Presolve", 0)
    set_optimizer_attribute(model, "Heuristics", 0)
# Presolve = 0,
#         PreCrush = 1,
#         Heuristics = 0,

    @variable(model, x[1:numArcs], Bin)
    @variable(model, theta >= -1e12)
    @objective(model, Min, theta)
    @constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    
#     @variable(model, 0 <= x <= 2.5, Int)
#     @variable(model, 0 <= y <= 2.5, Int)
#     @objective(model, Max, y)
#     println("\nM_set = ", M_set," \t P_set = ", P_set)
    cb_calls = Cint[]
    function my_callback_function(cb_data, cb_where::Cint)
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        iter += 1
#         println("\niter = ", iter)
#         println("\nM_set = ", M_set," \t P_set = ", P_set)
        # You can select where the callback is run
        if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
            return
        end
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
            if resultP[] != GRB_OPTIMAL
                return  # Solution is something other than optimal.
            end
        end
        # Before querying `callback_value`, you must call:
        Gurobi.load_callback_variable_primal(cb_data, cb_where)
        x_val = callback_value.(Ref(cb_data), x)
        theta_val = callback_value(cb_data, theta)
        P_set, M_set, lambda, pi_, inner_Obj = solveLP_findM(x_val)
        numPaths = length(P_set)
        numY = length(M_set)
#         println("x_val = ", x_val)
#         println("lambda = ", lambda, "\t pi_ = ", pi_)
#         println("\nM_set = ", M_set," \t P_set = ", P_set)
        # You can submit solver-independent MathOptInterface attributes such as
        # lazy constraints, user-cuts, and heuristic solutions.
#         if y_val - x_val > 1 + 1e-6
#             con = @build_constraint(y - x <= 1)
#             MOI.submit(model, MOI.LazyConstraint(cb_data), con)
#         elseif y_val + x_val > 3 + 1e-6
#             con = @build_constraint(y + x <= 3)
#             MOI.submit(model, MOI.LazyConstraint(cb_data), con)
#         end
#         lambda = [0.30103, 0.69897]
#         pi_ = 0.7895890626254753
        TOL = 10^(-12)
        
        if theta_val + TOL < inner_Obj 
        println("\niter = ", iter)
        println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
#             if x_val == [0,0,0]
                println("x_val = ", x_val)
                println("lambda = ", lambda, "\t pi_ = ", pi_)
                con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             if x_val == [1,0,0]
#                 println("x_val = ", x_val)
#                 println("lambda = ", lambda, "\t pi_ = ", pi_)
#                 con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             if x_val == [0,0,1]
#                 println("x_val = ", x_val)
#                 println("lambda = ", lambda, "\t pi_ = ", pi_)
#                 con = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println(sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 MOI.submit(model, MOI.LazyConstraint(cb_data),con)
#             end
#             println("Adding constraint")
#             con = @build_constraint(theta - sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) >= pi_)
#             println(" ", con)
    #         display(con)
#             MOI.submit(model, MOI.LazyConstraint(cb_data),con)
        end
#         if rand() < 0.1
#             # You can terminate the callback as follows:
#             GRBterminate(backend(model))
#         end
        return
    end
    # You _must_ set this parameter if using lazy constraints.
    MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
    MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
    println(model)
    optimize!(model)
    println(model)
    println("x = ", JuMP.value.(x))
    println("theta = ", JuMP.value.(theta))
# end
# solveMP()

In [ ]:
#USING REGULAR CONSTRAINTS TO VERIFY
using JuMP, Gurobi, Test, Combinatorics, LightGraphs

# function solveMP()
    ins = 1 #:10
    include("./Ins"*string(ins)*".jl")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY
    
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    
    global numArcs = length(arcs[:,1])
    global X_feas = EnumX(arcs, b_x, numArcs, d_x)
    global EnumY = EnumX(arcs, b_y, numArcs, d_y)
    global M_set = [EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    global numY = 1 
    global R = 10^6#0^6
    global P_set = [P[1]]

#     global x_now, current_Obj, y_now
    println("X_feas = ", X_feas)
    println("Starting M_set ", M_set, ", P_set = ", P_set)
    global Y_sol = []

    global numPaths = 1
    global newP = true
    global newM = true
    global iter = 0
    global tol = 10^(-6)
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
    
    model = direct_model(Gurobi.Optimizer())
    set_optimizer_attribute(model, "Presolve", 0)
#     set_optimizer_attribute(model, "Heuristics", 0)
# Presolve = 0,
#         PreCrush = 1,
#         Heuristics = 0,

    @variable(model, x[1:numArcs], Bin)
    @variable(model, theta >= -1e12)
    @objective(model, Min, theta)
    con = @constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
typeof(con)

In [ ]:
arcs = [1 4; 1 5; 1 12; 1 15; 1 16; 1 20; 1 25; 1 27; 1 28; 1 31; 1 44; 1 46; 1 49; 2 5; 2 6; 2 12; 2 24; 2 28; 2 29; 2 31; 2 34; 2 38; 2 40; 2 47; 2 48; 3 6; 3 7; 3 11; 3 13; 3 14; 3 15; 3 27; 3 28; 3 37; 3 39; 3 40; 3 46; 3 47; 3 50; 4 14; 4 16; 4 17; 4 24; 4 28; 5 6; 5 8; 5 9; 5 20; 5 21; 5 23; 5 26; 5 27; 5 32; 5 34; 5 35; 5 41; 5 43; 5 46; 5 49; 6 8; 6 9; 6 10; 6 11; 6 15; 6 17; 6 20; 6 25; 6 26; 6 29; 6 32; 6 36; 6 37; 6 38; 6 39; 6 40; 6 41; 7 3; 7 13; 7 16; 7 24; 7 25; 7 31; 7 43; 7 45; 7 47; 7 48; 7 49; 8 2; 8 6; 8 7; 8 9; 8 19; 8 20; 8 23; 8 24; 8 26; 8 33; 8 37; 8 43; 8 45; 8 50; 9 12; 9 13; 9 17; 9 22; 9 23; 9 31; 10 3; 10 14; 10 16; 10 22; 10 27; 10 28; 10 29; 10 32; 10 36; 10 41; 10 44; 10 48; 11 7; 11 9; 11 15; 11 17; 11 19; 11 22; 11 25; 11 26; 11 35; 11 37; 11 38; 11 40; 11 41; 11 45; 11 47; 12 6; 12 7; 12 8; 12 10; 12 15; 12 16; 12 28; 12 29; 12 38; 12 42; 12 43; 12 49; 12 50; 13 5; 13 11; 13 15; 13 16; 13 25; 13 33; 13 49; 14 2; 14 26; 14 28; 14 31; 14 36; 14 45; 14 49; 15 7; 15 10; 15 23; 15 26; 15 32; 15 35; 15 40; 16 9; 16 10; 16 20; 16 22; 16 24; 16 29; 16 34; 16 37; 16 44; 17 7; 17 8; 17 10; 17 11; 17 12; 17 15; 17 18; 17 20; 17 21; 17 25; 17 31; 17 39; 17 40; 17 41; 17 45; 18 2; 18 10; 18 17; 18 19; 18 21; 18 22; 18 26; 18 29; 18 30; 18 38; 18 39; 18 42; 18 43; 18 44; 18 45; 18 48; 19 2; 19 10; 19 18; 19 28; 19 31; 19 34; 19 38; 19 45; 19 46; 19 47; 19 48; 20 3; 20 4; 20 5; 20 8; 20 13; 20 18; 20 21; 20 32; 20 42; 20 43; 20 46; 20 47; 21 2; 21 10; 21 14; 21 16; 21 27; 21 28; 21 30; 21 31; 21 33; 21 34; 21 35; 21 43; 21 45; 22 4; 22 5; 22 11; 22 14; 22 15; 22 20; 22 23; 22 29; 22 39; 22 41; 23 2; 23 6; 23 10; 23 14; 23 21; 23 25; 23 30; 23 38; 23 39; 23 47; 23 50; 24 5; 24 6; 24 15; 24 16; 24 17; 24 18; 24 19; 24 27; 24 30; 24 42; 24 43; 24 46; 25 2; 25 6; 25 9; 25 12; 25 24; 25 26; 25 30; 25 31; 25 36; 25 39; 26 4; 26 5; 26 10; 26 23; 26 28; 26 34; 26 35; 26 39; 26 41; 26 44; 26 47; 27 4; 27 8; 27 12; 27 25; 27 26; 27 29; 27 36; 27 37; 27 39; 27 40; 27 50; 28 2; 28 7; 28 9; 28 10; 28 16; 28 19; 28 21; 28 22; 28 23; 28 26; 28 29; 28 30; 28 31; 28 35; 28 41; 28 47; 28 48; 28 50; 29 3; 29 6; 29 9; 29 15; 29 21; 29 22; 29 23; 29 30; 29 36; 29 45; 29 47; 29 50; 30 3; 30 6; 30 11; 30 12; 30 14; 30 18; 30 21; 30 25; 30 27; 30 29; 30 32; 30 33; 30 35; 30 38; 30 39; 30 47; 31 2; 31 5; 31 7; 31 8; 31 11; 31 12; 31 16; 31 20; 31 28; 31 40; 31 41; 31 42; 31 49; 32 12; 32 22; 32 24; 32 26; 32 35; 32 40; 33 2; 33 3; 33 6; 33 8; 33 9; 33 10; 33 12; 33 13; 33 17; 33 22; 33 25; 33 26; 33 27; 33 28; 33 34; 33 36; 33 38; 33 40; 33 42; 33 48; 34 4; 34 5; 34 12; 34 14; 34 18; 34 28; 34 31; 34 33; 34 36; 34 37; 34 38; 35 6; 35 8; 35 11; 35 12; 35 15; 35 16; 35 20; 35 22; 35 23; 35 30; 35 36; 35 44; 35 45; 35 47; 35 49; 36 2; 36 6; 36 7; 36 8; 36 12; 36 22; 36 24; 36 32; 36 39; 36 40; 36 47; 37 8; 37 15; 37 21; 37 22; 37 24; 37 27; 37 28; 37 30; 37 33; 37 34; 37 35; 37 36; 37 40; 37 46; 37 48; 38 2; 38 5; 38 10; 38 11; 38 17; 38 18; 38 20; 38 25; 38 26; 38 30; 38 39; 38 49; 39 2; 39 21; 39 23; 39 26; 39 27; 39 31; 39 34; 39 43; 39 45; 40 3; 40 7; 40 9; 40 11; 40 12; 40 19; 40 22; 40 24; 40 25; 40 34; 40 42; 40 45; 40 46; 40 49; 41 6; 41 9; 41 16; 41 20; 41 25; 41 26; 41 37; 42 3; 42 9; 42 15; 42 21; 42 22; 42 30; 42 34; 42 38; 42 44; 42 47; 43 3; 43 9; 43 11; 43 17; 43 19; 43 20; 43 27; 43 29; 43 36; 43 38; 43 42; 43 46; 43 47; 44 2; 44 3; 44 6; 44 11; 44 16; 44 22; 44 31; 44 32; 44 35; 44 41; 44 43; 44 45; 44 47; 45 5; 45 6; 45 7; 45 10; 45 17; 45 19; 45 21; 45 24; 45 25; 45 27; 45 29; 45 37; 45 39; 45 48; 45 49; 46 2; 46 4; 46 6; 46 7; 46 10; 46 11; 46 12; 46 21; 46 22; 46 23; 46 26; 46 28; 46 29; 46 38; 46 41; 47 9; 47 13; 47 14; 47 16; 47 20; 47 35; 47 36; 47 39; 47 40; 47 41; 47 43; 47 44; 47 45; 47 48; 47 50; 48 2; 48 6; 48 8; 48 10; 48 18; 48 22; 48 23; 48 24; 48 26; 48 34; 48 35; 48 38; 48 39; 48 42; 48 46; 49 4; 49 6; 49 15; 49 16; 49 25; 49 28; 49 31; 49 36; 49 41; 49 44; 49 50]
d_x = [4.0, 9.0, 10.0, 4.0, 2.0, 5.0, 9.0, 3.0, 2.0, 5.0, 4.0, 2.0, 2.0, 3.0, 3.0, 1.0, 1.0, 5.0, 5.0, 3.0, 4.0, 1.0, 5.0, 4.0, 7.0, 2.0, 3.0, 2.0, 10.0, 9.0, 10.0, 8.0, 8.0, 7.0, 7.0, 7.0, 6.0, 4.0, 9.0, 7.0, 6.0, 2.0, 3.0, 3.0, 3.0, 3.0, 7.0, 4.0, 4.0, 3.0, 2.0, 4.0, 1.0, 8.0, 2.0, 8.0, 8.0, 5.0, 9.0, 7.0, 5.0, 1.0, 6.0, 5.0, 1.0, 10.0, 7.0, 5.0, 10.0, 8.0, 6.0, 1.0, 9.0, 4.0, 9.0, 7.0, 8.0, 4.0, 4.0, 8.0, 8.0, 8.0, 4.0, 6.0, 9.0, 9.0, 2.0, 4.0, 10.0, 10.0, 4.0, 7.0, 5.0, 7.0, 10.0, 8.0, 6.0, 3.0, 8.0, 9.0, 7.0, 2.0, 6.0, 10.0, 1.0, 9.0, 8.0, 3.0, 5.0, 6.0, 7.0, 4.0, 10.0, 9.0, 7.0, 2.0, 1.0, 7.0, 2.0, 9.0, 2.0, 10.0, 7.0, 1.0, 6.0, 6.0, 7.0, 8.0, 5.0, 6.0, 3.0, 5.0, 3.0, 6.0, 3.0, 5.0, 3.0, 3.0, 4.0, 8.0, 7.0, 3.0, 6.0, 1.0, 8.0, 7.0, 7.0, 4.0, 10.0, 5.0, 6.0, 3.0, 5.0, 4.0, 7.0, 2.0, 4.0, 10.0, 4.0, 10.0, 10.0, 4.0, 3.0, 5.0, 9.0, 4.0, 10.0, 9.0, 6.0, 2.0, 1.0, 5.0, 1.0, 10.0, 10.0, 8.0, 8.0, 10.0, 9.0, 10.0, 2.0, 1.0, 2.0, 10.0, 10.0, 5.0, 9.0, 10.0, 5.0, 10.0, 2.0, 8.0, 10.0, 5.0, 8.0, 8.0, 8.0, 9.0, 8.0, 2.0, 3.0, 4.0, 4.0, 8.0, 6.0, 2.0, 2.0, 3.0, 8.0, 8.0, 2.0, 7.0, 3.0, 6.0, 7.0, 5.0, 3.0, 1.0, 6.0, 2.0, 7.0, 3.0, 2.0, 3.0, 7.0, 5.0, 9.0, 5.0, 10.0, 9.0, 6.0, 2.0, 6.0, 6.0, 7.0, 7.0, 10.0, 3.0, 4.0, 1.0, 3.0, 2.0, 5.0, 8.0, 2.0, 6.0, 6.0, 4.0, 10.0, 3.0, 3.0, 7.0, 10.0, 5.0, 2.0, 9.0, 5.0, 7.0, 7.0, 4.0, 6.0, 2.0, 5.0, 9.0, 6.0, 7.0, 9.0, 9.0, 8.0, 1.0, 10.0, 5.0, 7.0, 5.0, 5.0, 7.0, 3.0, 7.0, 8.0, 4.0, 10.0, 5.0, 3.0, 9.0, 3.0, 10.0, 2.0, 7.0, 1.0, 9.0, 4.0, 10.0, 3.0, 1.0, 1.0, 10.0, 5.0, 10.0, 8.0, 6.0, 7.0, 6.0, 4.0, 2.0, 10.0, 10.0, 6.0, 10.0, 8.0, 1.0, 4.0, 3.0, 3.0, 1.0, 1.0, 1.0, 2.0, 9.0, 9.0, 8.0, 3.0, 1.0, 4.0, 6.0, 3.0, 4.0, 8.0, 3.0, 2.0, 1.0, 10.0, 6.0, 3.0, 2.0, 1.0, 8.0, 4.0, 7.0, 9.0, 10.0, 4.0, 6.0, 7.0, 8.0, 8.0, 2.0, 6.0, 5.0, 9.0, 6.0, 2.0, 7.0, 10.0, 1.0, 10.0, 1.0, 4.0, 5.0, 7.0, 3.0, 4.0, 3.0, 5.0, 2.0, 6.0, 10.0, 4.0, 8.0, 3.0, 10.0, 7.0, 2.0, 6.0, 4.0, 7.0, 10.0, 1.0, 5.0, 8.0, 2.0, 2.0, 1.0, 5.0, 4.0, 10.0, 6.0, 4.0, 6.0, 6.0, 4.0, 5.0, 7.0, 6.0, 5.0, 10.0, 1.0, 7.0, 8.0, 4.0, 2.0, 3.0, 5.0, 7.0, 10.0, 4.0, 1.0, 4.0, 8.0, 1.0, 2.0, 8.0, 1.0, 7.0, 9.0, 4.0, 7.0, 4.0, 4.0, 1.0, 6.0, 10.0, 8.0, 2.0, 4.0, 5.0, 6.0, 7.0, 7.0, 9.0, 3.0, 3.0, 5.0, 7.0, 4.0, 7.0, 6.0, 5.0, 2.0, 4.0, 1.0, 2.0, 9.0, 3.0, 8.0, 8.0, 3.0, 9.0, 8.0, 1.0, 4.0, 2.0, 10.0, 7.0, 4.0, 1.0, 5.0, 6.0, 10.0, 8.0, 6.0, 7.0, 3.0, 10.0, 6.0, 9.0, 9.0, 5.0, 10.0, 2.0, 4.0, 7.0, 5.0, 1.0, 1.0, 10.0, 5.0, 8.0, 6.0, 7.0, 4.0, 3.0, 7.0, 5.0, 9.0, 3.0, 7.0, 6.0, 6.0, 1.0, 2.0, 6.0, 10.0, 8.0, 10.0, 4.0, 6.0, 10.0, 6.0, 7.0, 10.0, 6.0, 1.0, 8.0, 10.0, 7.0, 10.0, 5.0, 2.0, 3.0, 10.0, 3.0, 6.0, 7.0, 9.0, 10.0, 10.0, 3.0, 8.0, 10.0, 9.0, 8.0, 8.0, 8.0, 4.0, 4.0, 2.0, 8.0, 2.0, 8.0, 4.0, 4.0, 8.0, 1.0, 6.0, 7.0, 9.0, 8.0, 8.0, 6.0, 2.0, 3.0, 10.0, 5.0, 5.0, 9.0, 4.0, 2.0, 10.0, 5.0, 6.0, 4.0, 3.0, 4.0, 2.0, 6.0, 2.0, 3.0, 5.0, 5.0, 5.0, 2.0, 5.0, 2.0, 5.0, 3.0, 3.0, 9.0, 4.0, 9.0, 5.0, 9.0, 7.0, 1.0, 8.0, 8.0, 9.0, 3.0, 6.0, 3.0, 9.0, 7.0, 1.0, 2.0, 2.0, 4.0, 4.0, 8.0, 4.0, 9.0, 9.0, 2.0, 2.0, 5.0, 7.0, 8.0]
b_x = 10
d_y = [6.0, 4.0, 7.0, 10.0, 2.0, 7.0, 3.0, 9.0, 9.0, 8.0, 4.0, 10.0, 6.0, 9.0, 7.0, 1.0, 5.0, 1.0, 5.0, 9.0, 9.0, 1.0, 4.0, 2.0, 2.0, 8.0, 9.0, 6.0, 5.0, 1.0, 8.0, 1.0, 1.0, 1.0, 7.0, 8.0, 7.0, 2.0, 3.0, 7.0, 10.0, 6.0, 7.0, 1.0, 2.0, 5.0, 9.0, 1.0, 4.0, 10.0, 1.0, 5.0, 10.0, 10.0, 9.0, 8.0, 1.0, 5.0, 5.0, 3.0, 10.0, 2.0, 10.0, 10.0, 2.0, 10.0, 1.0, 8.0, 2.0, 4.0, 2.0, 10.0, 4.0, 7.0, 4.0, 9.0, 10.0, 2.0, 1.0, 3.0, 2.0, 4.0, 10.0, 1.0, 8.0, 3.0, 4.0, 7.0, 2.0, 2.0, 10.0, 3.0, 5.0, 2.0, 1.0, 6.0, 3.0, 4.0, 2.0, 1.0, 10.0, 2.0, 10.0, 7.0, 9.0, 5.0, 4.0, 7.0, 4.0, 6.0, 6.0, 4.0, 1.0, 8.0, 8.0, 9.0, 3.0, 6.0, 7.0, 2.0, 10.0, 3.0, 4.0, 6.0, 9.0, 5.0, 1.0, 6.0, 7.0, 9.0, 7.0, 2.0, 10.0, 6.0, 2.0, 6.0, 2.0, 6.0, 7.0, 6.0, 3.0, 6.0, 5.0, 4.0, 5.0, 10.0, 3.0, 6.0, 6.0, 1.0, 3.0, 7.0, 6.0, 3.0, 8.0, 10.0, 10.0, 8.0, 3.0, 10.0, 10.0, 3.0, 6.0, 7.0, 3.0, 2.0, 5.0, 3.0, 9.0, 4.0, 2.0, 5.0, 8.0, 2.0, 1.0, 8.0, 6.0, 7.0, 3.0, 9.0, 4.0, 4.0, 10.0, 1.0, 1.0, 5.0, 2.0, 8.0, 9.0, 7.0, 10.0, 2.0, 7.0, 10.0, 1.0, 1.0, 9.0, 9.0, 7.0, 7.0, 6.0, 1.0, 3.0, 2.0, 4.0, 5.0, 1.0, 2.0, 2.0, 10.0, 2.0, 10.0, 9.0, 10.0, 1.0, 8.0, 6.0, 4.0, 6.0, 10.0, 2.0, 10.0, 2.0, 10.0, 1.0, 1.0, 8.0, 9.0, 3.0, 6.0, 1.0, 1.0, 7.0, 1.0, 5.0, 5.0, 7.0, 3.0, 6.0, 5.0, 2.0, 7.0, 9.0, 5.0, 8.0, 5.0, 1.0, 10.0, 3.0, 10.0, 4.0, 10.0, 5.0, 10.0, 5.0, 7.0, 8.0, 7.0, 4.0, 4.0, 3.0, 2.0, 10.0, 10.0, 1.0, 9.0, 3.0, 9.0, 1.0, 8.0, 9.0, 8.0, 9.0, 2.0, 4.0, 5.0, 5.0, 5.0, 5.0, 6.0, 8.0, 9.0, 4.0, 6.0, 1.0, 8.0, 5.0, 2.0, 10.0, 8.0, 8.0, 4.0, 3.0, 4.0, 10.0, 9.0, 3.0, 10.0, 5.0, 6.0, 10.0, 4.0, 5.0, 10.0, 8.0, 10.0, 8.0, 8.0, 5.0, 2.0, 5.0, 4.0, 5.0, 9.0, 9.0, 1.0, 3.0, 10.0, 8.0, 2.0, 8.0, 2.0, 9.0, 1.0, 10.0, 8.0, 5.0, 5.0, 10.0, 9.0, 5.0, 1.0, 3.0, 5.0, 2.0, 9.0, 8.0, 7.0, 9.0, 8.0, 8.0, 8.0, 2.0, 1.0, 3.0, 6.0, 3.0, 4.0, 9.0, 3.0, 8.0, 4.0, 5.0, 5.0, 7.0, 4.0, 9.0, 4.0, 7.0, 4.0, 3.0, 5.0, 7.0, 10.0, 5.0, 5.0, 4.0, 2.0, 6.0, 3.0, 9.0, 5.0, 9.0, 4.0, 3.0, 4.0, 9.0, 4.0, 2.0, 9.0, 9.0, 6.0, 5.0, 2.0, 10.0, 10.0, 2.0, 8.0, 4.0, 10.0, 9.0, 10.0, 10.0, 6.0, 2.0, 3.0, 2.0, 2.0, 7.0, 1.0, 6.0, 2.0, 10.0, 9.0, 6.0, 5.0, 2.0, 6.0, 5.0, 2.0, 8.0, 8.0, 4.0, 10.0, 2.0, 1.0, 3.0, 7.0, 1.0, 10.0, 1.0, 6.0, 9.0, 4.0, 7.0, 1.0, 9.0, 6.0, 1.0, 5.0, 9.0, 6.0, 4.0, 7.0, 2.0, 2.0, 8.0, 4.0, 8.0, 10.0, 4.0, 4.0, 6.0, 7.0, 5.0, 10.0, 5.0, 5.0, 7.0, 10.0, 6.0, 6.0, 8.0, 8.0, 10.0, 10.0, 7.0, 5.0, 7.0, 6.0, 1.0, 6.0, 6.0, 9.0, 6.0, 2.0, 1.0, 10.0, 3.0, 9.0, 3.0, 8.0, 7.0, 10.0, 2.0, 8.0, 4.0, 2.0, 6.0, 6.0, 3.0, 1.0, 2.0, 6.0, 9.0, 6.0, 6.0, 4.0, 9.0, 8.0, 8.0, 3.0, 9.0, 4.0, 9.0, 9.0, 7.0, 9.0, 8.0, 2.0, 3.0, 8.0, 1.0, 1.0, 9.0, 8.0, 3.0, 6.0, 3.0, 1.0, 5.0, 9.0, 10.0, 9.0, 10.0, 10.0, 4.0, 4.0, 3.0, 2.0, 1.0, 4.0, 1.0, 8.0, 5.0, 4.0, 2.0, 1.0, 10.0, 10.0, 2.0, 3.0, 9.0, 5.0, 4.0, 2.0, 8.0, 7.0, 9.0, 4.0, 1.0, 4.0, 10.0, 4.0, 9.0, 1.0, 6.0, 6.0, 1.0, 7.0, 6.0, 5.0, 6.0, 8.0, 6.0, 5.0, 3.0, 3.0, 2.0, 1.0, 8.0, 10.0, 1.0, 2.0, 10.0, 3.0, 6.0, 9.0, 6.0, 2.0, 2.0, 8.0, 9.0, 2.0, 7.0, 1.0, 6.0, 9.0, 4.0, 8.0, 10.0, 8.0, 1.0, 6.0, 9.0, 2.0, 7.0, 10.0, 4.0, 9.0, 1.0, 4.0, 8.0, 3.0, 9.0]
b_y = 10
q = [0.76, 0.467, 0.125, 0.322, 0.556, 0.488, 0.532, 0.273, 0.877, 0.474, 0.374, 0.512, 0.595, 0.009, 0.449, 0.43, 0.83, 0.898, 0.413, 0.798, 0.791, 0.438, 0.952, 0.602, 0.336, 0.96, 0.508, 0.333, 0.752, 0.952, 0.469, 0.802, 0.137, 0.6, 0.857, 0.123, 0.905, 0.492, 0.394, 0.11, 0.314, 0.847, 0.554, 0.616, 0.431, 0.863, 0.234, 0.908, 0.661, 0.923, 0.572, 0.01, 0.507, 0.071, 0.15, 0.396, 0.116, 0.14, 0.969, 0.511, 0.635, 0.398, 0.06, 0.817, 0.447, 0.025, 0.628, 0.439, 0.264, 0.298, 0.251, 0.324, 0.518, 0.258, 0.673, 0.744, 0.896, 0.179, 0.58, 0.891, 0.612, 0.793, 0.142, 0.008, 0.378, 0.954, 0.958, 0.995, 0.501, 0.4, 0.698, 0.69, 0.551, 0.204, 0.255, 0.054, 0.699, 0.265, 0.304, 0.397, 0.89, 0.411, 0.128, 0.122, 0.901, 0.219, 0.537, 0.458, 0.71, 0.163, 0.69, 0.215, 0.147, 0.103, 0.937, 0.322, 0.449, 0.309, 0.835, 0.094, 0.979, 0.487, 0.704, 0.444, 0.639, 0.347, 0.838, 0.917, 0.409, 0.477, 0.406, 0.283, 0.479, 0.689, 0.431, 0.939, 0.614, 0.049, 0.407, 0.732, 0.81, 0.718, 0.519, 0.26, 0.739, 0.109, 0.746, 0.82, 0.598, 0.721, 0.884, 0.16, 0.958, 0.048, 0.213, 0.49, 0.224, 0.367, 0.734, 0.109, 0.076, 0.165, 0.32, 0.889, 0.887, 0.534, 0.729, 0.026, 0.614, 0.407, 0.201, 0.912, 0.141, 0.595, 0.408, 0.715, 0.097, 0.519, 0.427, 0.171, 0.824, 0.189, 0.377, 0.86, 0.583, 0.6, 0.548, 0.333, 0.961, 0.986, 0.636, 0.429, 0.155, 0.117, 0.824, 0.097, 0.04, 0.704, 0.96, 0.379, 0.137, 0.739, 0.457, 0.964, 0.188, 0.92, 0.655, 0.421, 0.084, 0.967, 0.15, 0.538, 0.201, 0.027, 0.635, 0.972, 0.144, 0.316, 0.381, 0.384, 0.547, 0.913, 0.874, 0.834, 0.896, 0.99, 0.123, 0.335, 0.021, 0.83, 0.605, 0.963, 0.67, 0.392, 0.421, 0.152, 0.259, 0.135, 0.003, 0.575, 0.828, 0.577, 0.578, 0.338, 0.196, 0.772, 0.306, 0.865, 0.028, 0.007, 0.402, 0.256, 0.474, 0.601, 0.854, 0.739, 0.319, 0.148, 0.782, 0.859, 0.352, 0.705, 0.977, 0.659, 0.034, 0.255, 0.398, 0.594, 0.615, 0.981, 0.973, 0.603, 0.483, 0.185, 0.232, 0.516, 0.66, 0.546, 0.057, 0.983, 0.89, 0.938, 0.413, 0.206, 0.95, 0.293, 0.634, 0.93, 0.666, 0.856, 0.069, 0.595, 0.398, 0.912, 0.876, 0.329, 0.023, 0.034, 0.451, 0.755, 0.022, 0.034, 0.337, 0.08, 0.092, 0.52, 0.9, 0.011, 0.812, 0.34, 0.085, 0.462, 0.854, 0.365, 0.758, 0.173, 0.113, 0.124, 0.304, 0.262, 0.274, 0.351, 0.367, 0.498, 0.176, 0.75, 0.85, 0.624, 0.502, 0.379, 0.567, 0.908, 0.676, 0.006, 0.454, 0.64, 0.582, 0.275, 0.754, 0.427, 0.542, 0.466, 0.601, 0.789, 0.889, 0.472, 0.5, 0.898, 0.296, 0.841, 0.071, 0.534, 0.03, 0.198, 0.522, 0.458, 0.527, 0.478, 0.289, 0.891, 0.04, 0.859, 0.012, 0.013, 0.891, 0.895, 0.647, 0.51, 0.439, 0.508, 0.121, 0.027, 0.312, 0.077, 0.769, 0.21, 0.21, 0.093, 0.265, 0.478, 0.187, 0.431, 0.797, 0.261, 0.456, 0.085, 0.234, 0.537, 0.204, 0.178, 0.946, 0.759, 0.792, 0.918, 0.687, 0.372, 0.614, 0.749, 0.592, 0.182, 0.053, 0.088, 0.757, 0.992, 0.661, 0.802, 0.85, 0.131, 0.96, 0.254, 0.361, 0.604, 0.088, 0.736, 0.077, 0.332, 0.74, 0.59, 0.701, 0.142, 0.135, 0.222, 0.958, 0.606, 0.09, 0.201, 0.742, 0.529, 0.48, 0.108, 0.594, 0.944, 0.711, 0.662, 0.633, 0.413, 0.941, 0.988, 0.678, 0.519, 0.915, 0.457, 0.847, 0.638, 0.134, 0.558, 0.851, 0.297, 0.451, 0.094, 0.028, 0.236, 0.823, 0.338, 0.711, 0.962, 0.5, 0.168, 0.8, 0.99, 0.694, 0.529, 0.781, 0.51, 0.666, 0.29, 0.848, 0.191, 0.476, 0.142, 0.926, 0.021, 0.578, 0.041, 0.884, 0.731, 0.746, 0.421, 0.817, 0.87, 0.157, 0.124, 0.43, 0.895, 0.265, 0.487, 0.856, 0.83, 0.526, 0.406, 0.925, 0.945, 0.516, 0.675, 0.07, 0.367, 0.038, 0.001, 0.48, 0.322, 0.087, 0.284, 0.646, 0.624, 0.444, 0.129, 0.683, 0.825, 0.845, 0.728, 0.156, 0.916, 0.752, 0.52, 0.5, 0.518, 0.401, 0.612, 0.307, 0.352, 0.494, 0.54, 0.857, 0.838, 0.219, 0.28, 0.146, 0.666, 0.774, 0.999, 0.362, 0.534, 0.525, 0.414, 0.615, 0.191, 0.908, 0.849, 0.025, 0.642, 0.029, 0.194, 0.947, 0.104, 0.594, 0.832, 0.836, 0.763, 0.22, 0.084, 0.258, 0.041, 0.201, 0.785, 0.828, 0.747, 0.944, 0.413, 0.022, 0.599, 0.135, 0.282, 0.946, 0.748, 0.359, 0.504, 0.261, 0.239, 0.968, 0.802, 0.761, 0.601, 0.008, 0.109, 0.423, 0.337, 0.012, 0.634, 0.349, 0.667, 0.958, 0.682, 0.861, 0.522, 0.706, 0.749, 0.079, 0.381, 0.6, 0.555, 0.17, 0.032, 0.52, 0.701, 0.37]
origin = 1
destination = 50


In [ ]:
Marc = [4, 33, 168, 265, 318, 468]
for a in Marc
    println(log10(1-q[a]))
end

In [27]:
include("./TestInstances/N20_d40_1.jl")
using JuMP, Gurobi, Test, Combinatorics, LightGraphs, TimerOutputs
println("Done loading packages")
include("./functionEnumFeasX.jl")
include("./functionFindQ.jl")
include("./functionGetCost_SP.jl")

include("./functionShortestPathBellmanFord.jl")
include("./functionGenMonitoring.jl")
include("./functionLP_findM.jl")
println("Done loading functions")
global numArcs = length(arcs[:,1])
#     global X_feas = EnumX(arcs, b_x, numArcs, d_x)
#     global EnumY = EnumX(arcs, b_y, numArcs, d_y)
M_start = findall(d_x.<=b_x)
global M_set = [[M_start[1]]]#[EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
global numY = 1 
global R = -sum(log10(1-q_) for q_ in q)  #10^6#0^6
    println("R = ", R)
path, gx = shortestPath_BellmanFord(q, arcs, numArcs, origin, destination)
arcsinPath = findall(path.>0)
global P_set = [arcsinPath]
#     global P_set = [P[1]]

#     global x_now, current_Obj, y_now

#     println("X_feas = ", X_feas)
#     println("Starting M_set ", M_set, ", P_set = ", P_set)

global y_sol = []
#     global 
global numPaths = 1
global newP = true
global newM = true
global iter = 0
model = direct_model(Gurobi.Optimizer())
set_optimizer_attribute(model, "OutputFlag", 0)
@variable(model, x[1:numArcs], Bin)
@variable(model, theta >= -1e6)
@objective(model, Min, theta)
@constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
# global M_set = [[1], [3], [12], [5, 57], [8, 103], [10], [7, 96, 129], [11, 134], [6, 28], [30], [9, 105], [15, 29], [4, 49], [32, 65], [33, 87], [35, 145], [48], [106], [34], [50], [73], [79], [110], [31, 70], [11, 124], [90, 104], [11, 20, 52], [11, 32, 33, 99], [12, 132], [22, 101], [74, 76, 82], [12, 26, 33, 102, 137], [11, 55, 100], [4, 108], [1, 4, 103, 134, 143], [1, 144], [1, 140], [1, 9, 94, 102], [4, 7, 9, 12], [76, 103, 131], [9, 67, 103], [12, 69, 82, 103], [29, 35, 76], [12, 34, 35], [2, 9], [9, 20, 98, 134], [7, 98, 102, 103, 143], [1, 12, 30, 98], [12, 98, 141], [12, 98, 135], [9, 12, 29, 98], [98, 127], [4, 12, 76, 98, 102], [9, 17, 103], [9, 12, 102, 103], [1, 13], [98, 102, 131], [17, 69, 98], [1, 98, 99], [4, 17, 102], [69, 85, 102], [4, 29, 102], [99, 102], [107], [9, 131], [9, 99], [12, 32, 98, 144], [12, 32, 98, 140], [6, 12, 98], [7, 33, 90], [7, 26, 32, 69], [7, 10], [1, 4, 69, 103], [1, 4, 74], [1, 4, 129], [1, 4, 90], [1, 4, 9, 20], [1, 4, 12, 42], [33, 41, 96, 126], [33, 41, 56, 70], [41, 66, 96], [1, 4, 35, 41], [4, 12, 15, 56, 96], [4, 44, 53], [4, 41, 43, 104], [1, 4, 12, 41, 43], [1, 12, 40], [1, 4, 33, 44], [33, 46], [1, 4, 12, 32, 56], [1, 4, 12, 70], [4, 28, 73], [4, 10, 28], [1, 12, 15, 53, 96], [1, 12, 15, 35], [1, 12, 15, 33, 134], [1, 12, 87], [1, 12, 15, 32, 102], [1, 12, 69, 76], [1, 3], [12, 31, 33, 98], [32, 35, 69, 98], [53, 98, 129], [9, 90, 98], [15, 35, 98, 102], [9, 30, 32, 98], [12, 32, 98, 99], [7, 9, 98, 102], [9, 10], [96, 98, 100], [29, 30], [12, 79], [29, 131], [4, 12, 99], [4, 7, 102, 103], [17, 30], [6, 102], [12, 17, 102], [12, 30, 102], [73, 102], [12, 100], [3, 11, 98], [4, 99, 103], [4, 79], [3, 4], [79, 103], [9, 30, 103], [3, 103], [30, 145]]
# global P_set = [[3, 39, 42], [12], [1, 28], [5, 57], [8, 103], [10, 129], [7, 96, 129], [11, 134], [6, 28, 66], [2, 30, 39], [9, 39, 105], [2, 15, 29, 39], [4, 49, 65], [2, 21, 32, 65, 66], [2, 33, 87], [2, 35, 145], [2, 31, 39, 42, 48], [2, 31, 39, 42, 51, 106], [2, 34, 39, 73, 100], [2, 31, 39, 50, 73, 100], [2, 31, 39, 51, 73, 112, 133], [2, 31, 39, 51, 79, 109], [2, 31, 39, 51, 90, 110], [2, 34, 39, 70, 99, 115], [2, 31, 39, 50, 70, 99, 115], [2, 31, 39, 51, 70, 108, 115], [8, 39, 73, 100], [2, 34, 39, 69, 79, 99], [2, 21, 34, 39, 63, 101, 124, 130], [8, 39, 69, 79, 99], [8, 39, 42, 48, 101, 131], [8, 21, 39, 63, 101, 124, 130], [2, 33, 39, 79], [2, 33, 39, 84, 124], [7, 39, 90], [9, 39, 42, 106], [9, 39, 73, 112, 133], [2, 23, 31, 39, 51, 90, 104], [9, 39, 79, 109], [9, 39, 90, 110], [9, 23, 39, 90, 104], [4, 39, 42, 48], [4, 39, 42, 51, 106], [10, 39, 124], [2, 32, 39, 42, 67], [11, 39, 73, 133], [5, 39, 42, 52], [2, 35, 39, 42, 137], [2, 18, 29, 39, 73], [11, 39, 69, 79, 132], [6, 39, 42, 67], [1, 23, 39, 90], [2, 34, 101, 134], [8, 101, 134], [2, 23, 34, 39, 90, 101, 130], [8, 23, 39, 90, 101, 130], [2, 33, 39, 82, 90], [10, 39, 61, 73, 126], [10, 39, 115, 127], [2, 20, 34, 39, 42, 52, 101, 130], [8, 20, 39, 42, 52, 101, 130], [2, 31, 39, 42, 51, 52, 107], [4, 39, 42, 51, 52, 107], [7, 39, 42, 91], [9, 39, 42, 48, 112, 131], [9, 39, 42, 52, 107], [8, 39, 42, 67, 99], [2, 31, 50, 101, 134], [2, 31, 51, 112, 134], [2, 31, 39, 42, 50, 67, 99], [2, 31, 39, 42, 51, 67, 108], [4, 39, 42, 50, 67, 99], [11, 39, 42, 67, 132], [5, 39, 42, 54, 67, 99], [4, 39, 42, 51, 67, 108], [9, 39, 42, 67, 108], [2, 21, 34, 65, 101, 130], [8, 21, 65, 101, 130], [2, 34, 69, 87, 101, 132], [2, 34, 39, 69, 79, 101, 132], [8, 69, 87, 101, 132], [2, 34, 39, 42, 67, 101, 132], [8, 39, 69, 79, 101, 132], [8, 39, 42, 67, 101, 132], [2, 22, 34, 69, 87, 101, 130], [2, 22, 34, 39, 69, 79, 101, 130], [8, 22, 69, 87, 101, 130], [2, 22, 34, 39, 42, 67, 101, 130], [8, 22, 39, 69, 79, 101, 130], [8, 22, 39, 42, 67, 101, 130], [2, 34, 76, 87, 100], [2, 34, 76, 85, 100, 134], [8, 76, 87, 100], [8, 76, 85, 100, 134], [2, 34, 39, 76, 79, 100], [2, 34, 39, 76, 82, 90, 100], [8, 39, 76, 79, 100], [8, 39, 76, 82, 90, 100], [2, 34, 65, 76, 82, 92, 100], [8, 65, 76, 82, 92, 100], [2, 34, 39, 42, 74, 100], [8, 39, 42, 74, 100], [2, 21, 34, 65, 102, 135], [8, 21, 65, 102, 135], [2, 34, 69, 87, 102, 140], [2, 26, 34, 102, 134, 135], [8, 69, 87, 102, 140], [8, 26, 102, 134, 135], [2, 34, 39, 42, 102, 137], [8, 39, 42, 102, 137], [2, 17, 34, 56, 71, 100, 134], [8, 17, 56, 71, 100, 134], [2, 13, 21, 34, 39, 60, 69, 71, 79, 100], [2, 13, 23, 34, 39, 71, 90, 100], [8, 13, 21, 39, 60, 69, 71, 79, 100], [8, 13, 23, 39, 71, 90, 100], [2, 34, 39, 77, 100, 124], [8, 39, 77, 100, 124], [2, 34, 39, 42, 48, 75, 100], [8, 39, 42, 48, 75, 100], [2, 17, 34, 39, 42, 52, 71, 100], [8, 17, 39, 42, 52, 71, 100], [2, 34, 39, 77, 100, 115, 127], [8, 39, 77, 100, 115, 127], [2, 21, 35, 65, 135], [2, 35, 39, 90, 141], [5, 56, 134], [5, 21, 55, 65, 104], [11, 22, 69, 87, 130], [5, 55, 87, 109], [11, 22, 39, 69, 79, 130], [5, 39, 42, 55, 106], [2, 34, 39, 90, 102, 141], [8, 39, 102, 115, 143], [8, 39, 102, 124, 144], [2, 31, 39, 42, 49, 60, 67], [1, 20, 39, 42, 52], [2, 31, 49, 65], [8, 39, 90, 102, 141], [2, 35, 39, 42, 67, 140], [2, 35, 87, 94, 141], [2, 35, 39, 42, 91, 141], [8, 87, 94, 102, 141], [4, 21, 51, 65, 112, 130], [8, 69, 87, 99], [8, 69, 85, 99, 134], [8, 39, 68, 73, 99], [8, 39, 69, 82, 90, 99], [6, 21, 65, 66], [8, 68, 76, 87, 99], [8, 68, 76, 85, 99, 134], [8, 21, 65, 66, 99], [8, 68, 77, 99, 129], [8, 30, 39, 98], [8, 31, 49, 65, 98], [8, 31, 49, 64, 98, 134], [8, 31, 51, 98, 113, 145], [8, 35, 98, 145], [8, 31, 51, 98, 112, 134], [8, 31, 39, 49, 61, 73, 98], [8, 15, 29, 39, 98], [8, 35, 39, 42, 98, 137], [2, 31, 39, 51, 105], [8, 102, 145], [4, 39, 51, 105], [3, 44, 96, 129], [5, 28, 56, 130], [8, 28, 66, 99], [3, 43, 56, 134], [3, 28, 41], [5, 49, 53, 65], [2, 28, 31, 51, 104], [6, 46, 67, 145], [8, 15, 39, 68, 71, 99], [3, 28, 44, 89], [5, 46, 52, 145], [3, 28, 43, 56, 130], [3, 13, 28, 40], [4, 51, 112, 134], [3, 15, 39, 40], [3, 14, 30, 39, 40], [3, 16, 40, 49, 65], [3, 46, 145], [8, 32, 69, 87, 98], [9, 112, 134], [7, 97, 134]]
# numY = length(M_set)
# numPaths = length(P_set)
global theta_val = -1e6
global inner_Obj = 1
while theta_val < inner_Obj - 1e-3
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y, y_sol
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
    global k

        iter += 1
    
        println("\niter = ", iter)
        optimize!(model)
        x_val = JuMP.value.(x)
        theta_val = JuMP.value.(theta)
        println("x = ", findall(x_val.==1))
        println("theta = ", theta_val)

        P_set, M_set, lambda, pi_, inner_Obj, y_sol = solveLP_findM(x_val)
        println("M = ", length(M_set), "\t P = ", length(P_set)) 
        println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
        lambda_pos = findall(lambda.>0)
        println("lambda = ", lambda[lambda_pos], "\t pi_ = ", pi_)
#         println("lambda ", lambda[lambda_pos[1]])
        numPaths = length(P_set)
        numY = length(M_set)

        TOL = 10^(-3)
        println("(theta_val - inner_Obj) = ", (theta_val - inner_Obj))

#             println("TOL = ", TOL)
        if (inner_Obj - theta_val) > TOL  
            @constraint(model,theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
        end
#         break
    end

end

Done loading packages
Done loading functions
R = 62.22451443576943

iter = 1
x = Int64[]
theta = -1.0e6
	1. Obj_LP = 1.0	[1.0]
	lambda = [1.0]
	pi = 1.0
	P_set = Array{Int64,1}[[3, 39, 42]]
[1]	1.0
1	interdict? -0.0	c_g 0.4596705252091263
	2. Obj_LP = 0.8976270912904415	[0.0, 1.0]
	lambda = [1.0]
	pi = 0.8976270912904415
	P_set = Array{Int64,1}[[3, 39, 42]]
[3]	1.0
3	interdict? -0.0	c_g 0.10237290870955855
	3. Obj_LP = 1.0	[0.0, 1.0]
	lambda = [0.0, 1.0]
	pi = 1.0
	P_set = Array{Int64,1}[[3, 39, 42], [12]]
[3]	1.0
3	interdict? -0.0	c_g 0.10237290870955855
	4. Obj_LP = 0.9273466741174003	[0.0, 0.709693, 0.290307]
	lambda = [0.709693, 0.290307]
	pi = 0.9273466741174003
	P_set = Array{Int64,1}[[3, 39, 42], [12]]
[3]	0.7096928943254307
[12]	0.2903071056745693
3	interdict? -0.0	c_g 0.0726533258825997
12	interdict? -0.0	c_g 0.07265332588259964
	5. Obj_LP = 0.9372626411569718	[0.136483, 0.612832, 0.250685]
	lambda = [0.612832, 0.250685, 0.136483]
	pi = 0.9372626411569717
	P_set = Array{Int64,

Excessive output truncated after 524289 bytes.

14681405855602003
[15, 29]	0.010157917173991832
[4, 49]	0.020731970100863143
[32, 65]	0.009949137564115328
[33, 87]	0.004724926372603722
[35, 145]	0.026550013701566105
[34]	0.0003589172325124997
[73]	0.026268831970013354
[79]	0.01644523309348394
[31, 70]	0.07812981423726087
[11, 124]	0.027291386584241517
[90, 104]	0.057986070488213554
[11, 32, 33, 99]	0.09031489124936226
[22, 101]	0.041184027467912374
[74, 76, 82]	0.03266422534281212
[12, 26, 33, 102, 137]	0.05398131767381676
1	interdict? -0.0	c_g 0.012800669068816271
3	interdict? -0.0	c_g 0.012887036132176986
4	interdict? -0.0	c_g 0.01288703613217698
5	interdict? -0.0	c_g 0.01288703613217698
6	interdict? -0.0	c_g 0.01288703613217698
7	interdict? -0.0	c_g 0.01288703613217698
8	interdict? -0.0	c_g 0.000273478383967128
9	interdict? -0.0	c_g 0.012887036132176972
10	interdict? -0.0	c_g 0.012887036132176984
11	interdict? -0.0	c_g 0.09490494506389871
12	interdict? -0.0	c_g 0.013509563451486342
15	interdict? -0.0	c_g 0.0009624012095499515
22	

InterruptException: [91mInterruptException:[39m

In [21]:
# println(length(q))
# println(q)
println("M_set = ", M_set)
println("P_set = ", P_set)

for path in P_set
    
   H_path = sum(log10(1-q[a]) for a in path) 
    println(path, "\t", H_path+1)
end

M_set = Array{Int64,1}[[1], [3], [12], [5, 57], [8, 103], [10], [7, 96, 129], [11, 134], [6, 28], [30], [9, 105], [15, 29], [4, 49], [32, 65], [33, 87], [35, 145], [48], [106], [34], [50], [73], [79], [110], [31, 70], [11, 124], [90, 104], [11, 20, 52], [11, 32, 33, 99], [12, 132], [22, 101], [74, 76, 82], [12, 26, 33, 102, 137], [11, 55, 100], [4, 108], [1, 4, 103, 134, 143], [1, 144], [1, 140], [1, 9, 94, 102], [4, 7, 9, 12], [76, 103, 131], [9, 67, 103], [12, 69, 82, 103], [29, 35, 76], [12, 34, 35], [2, 9], [9, 20, 98, 134], [7, 98, 102, 103, 143], [1, 12, 30, 98], [12, 98, 141], [12, 98, 135], [9, 12, 29, 98], [98, 127], [4, 12, 76, 98, 102], [9, 17, 103], [9, 12, 102, 103], [1, 13], [98, 102, 131], [17, 69, 98], [1, 98, 99], [4, 17, 102], [69, 85, 102], [4, 29, 102], [99, 102], [107], [9, 131], [9, 99], [12, 32, 98, 144], [12, 32, 98, 140], [6, 12, 98], [7, 33, 90], [7, 26, 32, 69], [7, 10], [1, 4, 69, 103], [1, 4, 74], [1, 4, 129], [1, 4, 90], [1, 4, 9, 20], [1, 4, 12, 42], [33,

In [8]:
using JuMP
using LightGraphs
using GraphPlot
include("./TestInstances/N20_d40_1.jl")
function plotGraph(edge)
    global origin, destination
#     n = maximum(edge)
#     println(n)
    n=destination
    g = SimpleDiGraph(n)
    label1 = collect(1:n)
    for i = 1:length(edge[:,1])
        add_edge!(g, edge[i,1], edge[i,2]);
    end
    gplot(g, layout=circular_layout, nodelabel=label1)
#     kruskal_mst(g, ones(length(edge[:,1])))
#     println(is_strongly_connected(g))
end
plotGraph(arcs)
global N = destination
function gx_bound(c, c_g, edge, source_sink)
    global  N, origin, destination
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    if source_sink == true
        start_node = edge[:,1]
        end_node = edge[:,2]
        origin = 1
        destination = N
    else 
        start_node = edge[:,2]
        end_node = edge[:,1]
        origin = N
        destination = 1
    end
    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    Len = length(edge[:,1])
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end
    
    origin = 1
    destination = N
#     println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
#     println("Minimum Spanning Tree =", pred)
#     println("Node label = " ,label)

    return y, gx, SP, label
end

gx_bound (generic function with 1 method)

In [9]:
numArcs = length(arcs[:,1])
c_g = ones(numArcs)
c = c_g
source_sink = true
y, gx, SP, label = gx_bound(c, c_g, arcs, source_sink)
println(label)
source_sink = false
y, gx, SP, label = gx_bound(c, c_g, arcs, source_sink)
println(label)

[0.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0]
[1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 0.0]
